

```
# This is formatted as code
```

# Динамическое программирование

## Числа Фибоначчи

In [25]:
count = 0

In [6]:
def fib(n):
    global count
    count = count + 1
    if n <= 1:
        return 1
    return fib(n-1) + fib(n-2)

In [12]:
[fib(n) for n in range(11)]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

range(0, 10)

In [27]:
count

10

Какая сложность? 

<!-- $O(2^n)$ -->


## top-down
У нас уже есть рекурентная формула, осталось добавить мемоизацию (memoization)

In [18]:
fibs = {}
def fib(n):
    if n <= 1:
        return 1
    global count
    count = count + 1
    if n not in fibs:
        fibs[n] = fib(n-1) + fib(n-2)
    return fibs[n]

In [20]:
[fib(n) for n in range(10)]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

Сложность $O(n)$ операций и $O(n)$ памяти

## bottom-up
Если мы знаем `fib(n-2)` и `fib(n-1)`, мы можем тривиально вычислить `fib(n)`

In [23]:
def fib(n):
    global count
    count = count + 1
    fibs = [0, 1]
    for i in range(2,n+1):
        fibs.append(fibs[i-1] + fibs[i-2])
    return fibs[n]

In [26]:
[fib(n) for n in range(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

Сложность $O(n)$ операций и $O(n)$ памяти

## Уменьшение потребления памяти

После вычисления `fib(n)` мы больше не используем `fib(k)`, $k \leq n-1$. Нам достаточно хранить `fib(n)` и `fib(n-1)`

In [ ]:
def fib(n):
    a = 0 # fib(i-2)
    b = 1 # fib(i-1)
    for i in range(2,n+1):
        a, b = b, a+b
    return b

In [ ]:
[fib(n) for n in range(10)]

[1, 1, 1, 2, 3, 5, 8, 13, 21, 34]

Сложность $O(n)$ операций и $O(1)$ памяти

# Размен
Есть неограниченный запас монет номиналами $S_1, \ldots, S_M$. Сколькими способами можно составить сумму N из них?

In [14]:
def coins(S, n, m = 0):
    if n == 0:
        return 1
    if n < 0:
        return 0
    if m == len(S):
        return 0
    return coins(S, n, m+1) + coins(S, n-S[m], m)

In [16]:
coins([1,2], 99)

50

## top-down

In [ ]:
def coins(S, n, m = 0, tmp = {}):
    if n == 0:
        return 1
    if n < 0:
        return 0
    if m == len(S):
        return 0
    if (n, m) not in tmp:
        tmp[(n, m)] = coins(S, n, m+1, tmp) + coins(S, n-S[m], m, tmp)
    return tmp[(n, m)]

In [ ]:
coins([1,2,5,10], 99)

2090

## bottom-up
Выберем такой порядок вычисления:

1. `coins(S, n, len(S)-1) for n in range(N+1)`
2. `coins(S, n, len(S)-2) for n in range(N+1)`
3. $\ldots$
4. `coins(S, n, 0) for n in range(N+1)`

Для вычисления `coins(S, N, M)` мы должны знать `coins(S, n, M) for n in range(N)` и `coins(S, N, M + 1)`

In [ ]:
def coins(S, n):
    table = [1] + [0]*n
    for coin in S[::-1]:
        for j in range(coin, n+1):
            table[j] = table[j] + table[j - coin]
    return table[n]

In [ ]:
coins([1,2,5,10], 99)

2090

# Упаковка рюкзака
Есть вещи с ценностью $v_1, \ldots, v_N$ и весом $w_1, \ldots, w_N$. Найти набор вещей общим весом не более $W$ с максимальной суммарной ценностью.

In [ ]:
def knapsack(v, w, W, m = 0):
    if m == len(w):
        return 0
    if w[m] > W:
        return knapsack(v, w, W, m+1)
    return max(knapsack(v, w, W, m+1), knapsack(v, w, W-w[m], m+1) + v[m])

In [ ]:
knapsack([7, 9, 5, 12, 14, 6, 12], [3, 4, 2, 6, 7, 3, 5], 15)

34

# Наибольшая общая подпоследовательность
Даны две строки $S$ и $T$. Найти наибольшее $k$, такое что существуют $i_1 < \ldots < i_k$ и $j_1 < \ldots < j_k$, такие что $S_{i_t} = T_{j_t}$.

In [ ]:
def lcs(S, T): # Longest Common Subsequence
    def do_lcs(n, m): # S[:n], T[:m]
        if n == 0 or m == 0:
            return 0
        if S[n-1] == T[m-1]:
            return 1 + do_lcs(n-1,m-1)
        return max(do_lcs(n-1, m), do_lcs(n, m-1))
    return do_lcs(len(S), len(T))

In [ ]:
lcs('ABCAC', 'ABCAC')

5

In [ ]:
lcs('ABAZDC', 'BACBAD')

4

# Кратчайшие пути в графе
Дан ориентрованный взвешенный граф, вес ребра из $i$ в $j$ равен $d_{ij}$. Найти длины кратчайших путей между каждой парой вершин $\overline{d}_{ij}$.

$d_{ij}^k = \min \left(d_{ij}^{k-1}, d_{ik}^{k-1} + d_{kj}^{k-1}\right)$

In [ ]:
def distances(d):
    N = len(d)
    for k in range(N):
        for i in range(N):
            for j in range(N):
                d[i][j] = min(d[i][j], d[i][k] + d[k][j])
    return d

In [ ]:
distances([[0,3,5], [2,0,1], [1,2,0]])

[[0, 3, 4], [2, 0, 1], [1, 2, 0]]